🚨🚨🚨 極度危險警告：此腳本會永久清空 Google 雲端的所有檔案 🚨🚨🚨

👉 請務必：
   1. **再三確認** 您真的要清空雲端硬碟中所有您擁有的項目。
   2. **強烈建議** 在執行此腳本前 **完整備份** 您的 Google 雲端資料。
   3. 了解此操作的 **永久性後果**。

除非您完全明白此腳本的作用並願意承擔資料遺失的風險，否則 **請勿執行**。

In [ ]:
#@markdown ⬅ 點擊執行按鈕開始清除
#@markdown ---
#@markdown <br><br>
#@title 清空雲端硬碟 { display-mode: "form" }
操作確認 = "" #@param {type:"string", placeholder:"輸入您的 email 以確認操作"}
#@markdown <br><br>
# -*- coding: utf-8 -*-
"""
🚨🚨🚨 極度危險警告：此腳本會清空 Google 雲端的所有檔案 🚨🚨🚨

此腳本會執行以下 **無法復原** 的操作：
1. 找出您帳號所擁有的 **所有** 檔案與資料夾。
2. 將找到的所有項目 **全部移至垃圾桶**。
3. **永久刪除** 您 Google 雲端垃圾桶裡的所有項目。

👉 請務必：
   1. **再三確認** 您真的要清空雲端硬碟中所有您擁有的項目。
   2. **強烈建議** 在執行此腳本前 **完整備份** 您的 Google 雲端資料。
   3. 了解此操作的 **永久性後果**。

除非您完全明白此腳本的作用並願意承擔資料遺失的風險，否則 **請勿執行**。
"""

# --- 安裝與載入套件 ---
# !pip install --quiet --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib tqdm

# 變數名稱轉換
EMAIL_VERIFY = 操作確認

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth import default
from tqdm.notebook import tqdm
import time
import sys

# --- 全域變數 ---
drive_service = None
user_email = ''
deleted_count = 0 # Renamed from 'deleted'
failed_count = 0 # Renamed from 'failed'

# --- 輔助函數 ---

def authenticate_and_build_service():
    """處理 Colab 驗證並建立 Drive API v3 服務"""
    global drive_service, user_email
    try:
        print("⏳ 正在請求 Google Drive 授權...")
        auth.authenticate_user()
        creds, _ = default()
        # 使用 v3 API
        drive_service = build('drive', 'v3', credentials=creds, cache_discovery=False)
        print("✅ 授權成功，正在建立 Drive v3 服務...")
        # 取得使用者資訊 (需要 v3)
        about = drive_service.about().get(fields="user(emailAddress)").execute()
        user_email = about['user']['emailAddress']
        print(f"✅ Drive 服務建立成功，目前使用者: {user_email}")
        return True
    except Exception as e:
        print(f"❌ 驗證或建立服務失敗: {e}")
        return False

def list_all_owned_items(): # Renamed to reflect files and folders
    """列出帳號所擁有的所有檔案與資料夾 ID (使用 v3 API)"""
    global drive_service
    item_ids = []
    page_token = None
    print("🔍 正在搜尋您擁有的所有檔案與資料夾...")
    pbar = tqdm(desc="搜尋進度", unit=" 頁")
    while True:
        try:
            param = {
                'q': "'me' in owners and trashed = false", # 只找未在垃圾桶的
                'pageSize': 1000, # v3 使用 pageSize
                'fields': "nextPageToken, files(id)", # v3 需要指定 fields
                'pageToken': page_token,
            }
            response = drive_service.files().list(**param).execute()
            files = response.get('files', []) # v3 使用 'files'
            item_ids.extend([f['id'] for f in files])
            page_token = response.get('nextPageToken')
            pbar.update(1)
            if not page_token:
                break
            time.sleep(0.1) # 稍微延遲避免過快請求
        except HttpError as error:
            print(f'❌ 搜尋過程中發生錯誤: {error}')
            # 即使出錯也返回目前找到的，讓使用者決定是否繼續
            break
        except Exception as e:
            print(f'❌ 搜尋過程中發生非預期錯誤: {e}')
            break
    pbar.close()
    print(f"📦 共找到 {len(item_ids)} 個您帳號擁有的項目 (檔案與資料夾)。")
    return item_ids

def trash_items_with_progress(item_ids): # Renamed
    """將指定的項目 ID 移至垃圾桶 (使用 v3 API)"""
    global drive_service, deleted_count, failed_count
    if not item_ids:
        print("🤷 沒有需要移至垃圾桶的項目。")
        return

    print(f"🗑️ 準備將 {len(item_ids)} 個項目移至垃圾桶...")
    for item_id in tqdm(item_ids, desc="移至垃圾桶", unit="個"):
        try:
            # v3 使用 update 將 trashed 設為 true
            drive_service.files().update(fileId=item_id, body={'trashed': True}).execute()
            deleted_count += 1
            time.sleep(0.1)  # 防止 rate limit
        except HttpError as error:
            failed_count += 1
            # 檢查是否為找不到檔案的錯誤 (可能已被刪除或權限變更)
            if error.resp.status == 404:
                 tqdm.write(f'⚠️ 移至垃圾桶失敗：找不到項目 {item_id} (可能已被刪除或權限變更)')
            else:
                 tqdm.write(f'⚠️ 移至垃圾桶失敗：{item_id} - {error}')
        except Exception as e:
            failed_count += 1
            tqdm.write(f'⚠️ 移至垃圾桶失敗：{item_id} - {e}')

def empty_trash():
    """清空 Google Drive 垃圾桶 (使用 v3 API)"""
    global drive_service
    try:
        print("🗑️  正在清空垃圾桶...")
        drive_service.files().emptyTrash().execute()
        print("✅ 垃圾桶已清空")
        return True
    except HttpError as error:
         # 如果垃圾桶已是空的，API 可能會回 404 或其他錯誤，這不算失敗
         # 但這裡為了簡單起見，仍將其視為警告
         print(f'⚠️ 清空垃圾桶時發生 API 錯誤 (可能是空的或權限問題): {error}')
         return False
    except Exception as e:
        print(f'❌ 清空垃圾桶時發生非預期錯誤：{e}')
        return False

# --- 主執行流程 ---

if __name__ == "__main__":
    # 1. 驗證並取得使用者 Email
    if not authenticate_and_build_service():
        sys.exit("❌ 無法繼續執行，請檢查驗證問題。")

    # 檢查 EMAIL_VERIFY 參數是否與獲取的 user_email 匹配
    # 確保 EMAIL_VERIFY 變數存在 (由 #@param 定義)
    if 'EMAIL_VERIFY' not in globals():
         print("\n❌ 錯誤：找不到 EMAIL_VERIFY 參數。請確保 Colab 表單已正確設定。")
         sys.exit("腳本已中止。")

    # 進行比較
    if EMAIL_VERIFY.strip() != user_email:
        print(f"\n🚫 操作已取消。表單中輸入的 Email ('{EMAIL_VERIFY}') 與偵測到的使用者 Email ('{user_email}') 不符或未輸入。")
        print("請確認您登入的帳號。")
        sys.exit("腳本已中止。")

    # --- 確認通過，開始執行 ---
    print("\n✅ Email 確認成功，開始執行清空流程...")

    # 3. 第一次清空垃圾桶 (清除舊有項目)
    print("\n--- 步驟 1/3：清空現有垃圾桶 (若有) ---")
    empty_trash()

    # 4. 列出所有擁有的項目
    print("\n--- 步驟 2/3：搜尋並移動您的項目至垃圾桶 ---")
    all_items = list_all_owned_items()

    # 5. 將項目移至垃圾桶
    trash_items_with_progress(all_items)
    print(f"📊 移動結果：成功移入 {deleted_count} 個，失敗 {failed_count} 個")

    # 6. 第二次清空垃圾桶 (永久刪除)
    print("\n--- 步驟 3/3：永久清空垃圾桶 ---")
    if deleted_count > 0 or failed_count > 0: # 只有在移動過項目或有失敗時才再次清空
       empty_trash()
    else:
       print("🤷 沒有項目被移至垃圾桶，無需再次清空。")

    print("\n🏁 ========== Google Drive 清空腳本執行完畢 ========== 🏁")
    print(f"最終結果：{deleted_count} 個項目被永久刪除，{failed_count} 個項目移動失敗。")


# 腳本說明：清空 Google 雲端硬碟

## 🚨🚨🚨 極度危險警告 🚨🚨🚨

**此腳本會永久刪除您 Google 雲端硬碟中所有您擁有的檔案和資料夾！操作無法復原！**

## 用途

*   此腳本旨在**完全清空**執行腳本的 Google 帳號在 Google Drive 中**所擁有的全部內容**。
*   它會分三步執行：
    1.  （可選）先清空一次垃圾桶，以處理先前已刪除的項目。
    2.  找出執行者帳號所擁有的**所有**檔案與資料夾（無論在哪個位置），並將它們**全部移至垃圾桶**。
    3.  **再次清空垃圾桶**，將剛剛移入的所有項目**永久刪除**。
*   包含一個**Email 確認機制**，要求使用者輸入自己的 Email，作為執行此毀滅性操作前的最後一道防線。

## 適用情境

*   需要徹底重設 Google Drive 空間，刪除所有個人擁有的內容時（**極少數情況**）。
*   在**已完成完整且可靠的備份**後，需要釋放帳號所有儲存空間時。
*   **強烈不建議**在常規情況下使用此腳本。

## 權限要求

*   **執行者**：執行此腳本的 Google 帳號需要：
    *   完整的 Google Drive 讀寫權限 (`https://www.googleapis.com/auth/drive`)，以便能夠：
        *   列出帳號擁有的所有檔案和資料夾。
        *   修改檔案和資料夾的狀態（將其移至垃圾桶）。
        *   清空帳號的垃圾桶。

## 警告說明

*   **永久刪除且無法復原**：這是此腳本最需要強調的警告。一旦執行完成，所有被移入垃圾桶並被清空的項目將**永久消失**，Google 也無法協助復原。
*   **清空所有擁有項目**：腳本會搜尋並刪除**您帳號所擁有的所有項目**，無論它們位於「我的雲端硬碟」還是您有權限修改的「共用雲端硬碟」中（只要您是擁有者）。它**不會**刪除僅與您共享、但您不是擁有者的項目。
*   **資料遺失風險**：執行此腳本等同於格式化您的 Google Drive。所有未備份的資料將會遺失。
*   **務必備份**：在考慮執行此腳本之前，**必須**確保您已將所有重要資料完整備份到 Google Drive 之外的其他地方。
*   **Email 確認**：請務必在執行前，於「操作確認」欄位**正確輸入您當前登入 Colab/執行腳本的 Google 帳號 Email**。這是防止意外啟動此腳本的關鍵步驟。輸入錯誤或留空將導致腳本中止。
*   **執行時間**：如果您的雲端硬碟中有大量項目，搜尋、移動和清空過程可能需要很長時間。
*   **API 限制**：處理極大量項目時，仍可能遇到 Google Drive API 的速率限制或超時，可能導致部分項目移動失敗，但腳本仍會嘗試清空垃圾桶。

**再次強調：除非您完全理解後果並已做好萬全準備，否則絕對不要執行此腳本。**